# implementing loss value calculation 

In [1]:
import numpy as np
import math

# categorical cross entropy loss

In [2]:
predicted=[0.7,0.2,0.1]
actual=[1,0,0]

loss=0
for i in range(len(predicted)):
    loss+=(math.log(predicted[i])*actual[i])

loss*=-1
print(loss)

0.35667494393873245


In [3]:
print(math.log(1))
print(math.log(0.9))
print(math.log(0.5))
print(math.log(0.3))

0.0
-0.10536051565782628
-0.6931471805599453
-1.2039728043259361


In [4]:
(float('inf')*0)

nan

In [5]:
np.log10(9)

0.9542425094393249

In [6]:
np.log(9)

2.1972245773362196

In [7]:
math.e**2.1972425094393249   # ==> e**9 =2.xxx

9.00016139037498

# lets see how to calculte the loss in batches

In [8]:
batch_outputs=[
    [0.7,0.2,0.1],
    [0.1,0.5,0.4],
    [0.02,0.9,0.08]
]  #this is a matrix of output probabilites by the model

actual_output=[0,1,1]  #this is the value of the index
chosen_output=[]
for act,pred in zip(actual_output,batch_outputs):
    chosen_output.append(pred[act])

print(chosen_output)

[0.7, 0.5, 0.9]


In [9]:
empty_lists=[]
for pred,act in zip(batch_outputs,actual_output):
    print(pred[act])
    empty_lists.append(pred[act])

print(f"new one:{empty_lists} ")

0.7
0.5
0.9
new one:[0.7, 0.5, 0.9] 


In [10]:
# lets try using numpy for this 
array_outputs=np.array(batch_outputs)
chosen_values=array_outputs[range(len(array_outputs)),actual_output]
print(chosen_values)

[0.7 0.5 0.9]


In [11]:
#from this lets calculate the categorical cross entropy
loss=[]
for i in chosen_values:
    loss.append(-math.log(i))
print(loss)
print(np.mean(loss))

[0.35667494393873245, 0.6931471805599453, 0.10536051565782628]
0.38506088005216804


In [12]:
np.mean([20,12,90])

40.666666666666664

# implementing the calculation of 2D-actual values

In [13]:
print(batch_outputs)

[[0.7, 0.2, 0.1], [0.1, 0.5, 0.4], [0.02, 0.9, 0.08]]


In [14]:
output_softmax=np.array(batch_outputs)
actual_values=[
    [1,0,0],
    [0,1,0],
    [0,1,0]
]

actual_values_a=np.array(actual_values)
loss=[]
#lets calculate the values here
for pred,act in zip(output_softmax,actual_values):
    temp_scaler=np.sum(pred*act)
    loss.append(-math.log(temp_scaler))

In [15]:
t=np.array([1,2,3])
t2=np.array([4,5,6])
t*t2

array([ 4, 10, 18])

In [16]:
np.mean(loss)

0.38506088005216804

# all inclusive way of calculting the cross entropy loss

In [17]:
len(actual_values_a.shape),len(np.array([1,2,3]).shape)

(2, 1)

In [18]:
def cross_entropy_loss(predicted_values,actual_values):
    if len(actual_values.shape)==2:
        correctness=np.sum(predicted_values*actual_values,axis=1)
    
    elif len(actual_values.shape)==1:
        #this is is the actual values is given in actual values rather than one hot encoded
        correctness=predicted_values[range(len(predicted_values)),actual_values]
    
    else:
        raise Exception("check for the shape again.")
    loss=[]
    
    # for i in correctness:
    #     loss.append(-math.log(i))
    loss=-np.log(correctness)
    avg_loss=np.mean(loss)
    return loss,avg_loss

In [19]:
cross_entropy_loss(actual_values_a,output_softmax)

(array([0.35667494, 0.69314718, 0.10536052]), 0.38506088005216804)

In [20]:
np.exp(-np.inf)

0.0

In [21]:
temp=np.array([
    [0,0,0],
    [1,1,1],
    [1,0,1]
])
np.clip(temp,1e-7,1-1e-7)

array([[1.000000e-07, 1.000000e-07, 1.000000e-07],
       [9.999999e-01, 9.999999e-01, 9.999999e-01],
       [9.999999e-01, 1.000000e-07, 9.999999e-01]])

# lets make a class

In [22]:
class Loss:
    def calculate(self,y_pred,y_true):
        self.loss=self.forward(y_pred,y_true)
        self.avg_loss=np.mean(self.loss)
        return self.avg_loss

In [23]:
class Categorical_loss_entropy(Loss):
    def __init__(self):
        super().__init__()
    def forward(self,y_pred,y_true):
        y_pred_cliped=np.clip(y_pred,1e-7,1-1e-7)
        #check for the type of actual values
        if len(y_true.shape)==2:
            correctness=np.sum(y_pred_cliped*y_true,axis=1)
            
        elif len(actual_values.shape==1):
            correctness=y_pred_cliped[range(len(y_pred_cliped)),y_true]
        else:
            raise Except("there is an error with the shape of the input.")
            
        self.loss=-np.log(correctness)
        return self.loss

In [24]:
loss_fn=Categorical_loss_entropy()

In [25]:
loss_fn.calculate(actual_output,output_softmax)

0.4431786936053461

# train a model

In [26]:
from dense import Dense
from activation import Relu,Softmax
from loss import Categorical_loss_entropy
from nnfs.datasets import spiral_data

## define the architecture of the model

In [27]:
x,y=spiral_data(samples=1000,classes=3)

#specify the first hidden layer
dense1=Dense(n_inputs=2,n_neurons=3)
act1=Relu()
dense2=Dense(n_inputs=3,n_neurons=3)  #the last shape is 3 because the outputs has 3 classes 
act2=Softmax()
loss=Categorical_loss_entropy()

# lets train--feed forward and evaluated it

In [28]:
dense1.forward(x)

act1.forward(dense1.outputs)

dense2.forward(act1.output)

act2.forward(dense2.outputs)

final_loss=loss.calculate(act2.outputs,y)

In [29]:
final_loss

1.098622618792298

In [30]:
print(act2.outputs[:5])

[[0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333332 0.33333343 0.33333325]
 [0.33333336 0.33333332 0.33333331]
 [0.33333339 0.33333332 0.33333329]]


# calculating the accuracy

## if the target is indexed

In [31]:
temp=np.array([
               [12,23,34],
               [34,54,23],
               [65,62,61]
              ])
class_targets=[2,0,0]

pred=np.argmax(temp,axis=1)

In [33]:
# lets calculate the accuracy
accuracy=np.mean(class_targets==pred)
print(accuracy)

0.6666666666666666


## if the target is one hot encoded

In [34]:
class_targets_mat=np.array([
    [0,0,1],
    [0,1,0],
    [1,0,0]
])

y_pred=np.array([
    [0.2,0.1,0.7],
    [0.8,0.1,0.1],
    [0.5,0.4,0.1]
])

acc=0
new_class_targets=[]

#we arrange the class targets
if len(class_targets_mat.shape)==2:
    new_class_targets=np.argmax(class_targets_mat,axis=1)

#lets extract the index of the predicted by the volume
predictions=np.argmax(y_pred,axis=1)

In [35]:
# now lets calculate the accuracy
accuracy_1=np.mean(new_class_targets==predictions)
print(f"the accuracy is:{accuracy_1} ")

the accuracy is:0.6666666666666666 
